In [ ]:
import sympy
import numpy as np
import matplotlib.pyplot as plt
import dill
dill.settings["recurse"] = True
from tqdm.notebook import tqdm
import plotly
from triangle import triangulate

from utils import (
    get_middle_indices
)

from scipy.sparse import lil_matrix
from scipy.sparse.linalg import spsolve

In [ ]:
weak_form_functional_lambdified = dill.load(open("../calculations/macro_cubic_hct_weak_form_functional_xyxy_symbolic", "rb"))
weak_form_right_part_lambdified = dill.load(open("../calculations/macro_cubic_hct_weak_form_right_part_symbolic", "rb"))
interpolation_function =          dill.load(open("../calculations/macro_cubic_hct_basis_symbolic", "rb"))
interpolation_function_normal =   dill.load(open("../calculations/macro_cubic_hct_basis_normal_symbolic", "rb"))
mapping_function_lambdified =     dill.load(open("../calculations/macro_cubic_hct_mapping_function_symbolic", "rb"))

## Get mesh

In [ ]:
t = np.linspace(0 + np.pi/4, 2*np.pi + np.pi/4, 10, endpoint=False)
points = np.stack((np.cos(t), np.sin(t))).T

seg = np.array(
    [[j, j + 1] for j in range(points.shape[0] - 1)] + [[points.shape[0] - 1, 0]]
)

In [ ]:
cndt = triangulate({"vertices": points, "segments": seg}, opts="pq30a0.2neo2s")
points = cndt["vertices"]
triangles = cndt["triangles"]
neighbours = cndt["neighbors"]
edges = cndt["edges"]
vertex_marker_is_boundary = cndt["vertex_markers"].squeeze()
edge_marker_is_boundary = cndt["edge_markers"].squeeze()
print(
    f"Total number of points:\t\t{points.shape[0]}\nTotal number of elements:\t{triangles.shape[0]}"
)

In [ ]:
is_middle = get_middle_indices(points.shape[0], triangles)

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(7, 7))
ax.axis("equal")

cl = ["black", "brown"]

for eidx, edge in enumerate(edges):
    ax.plot(
        points[edge][:, 0],
        points[edge][:, 1],
        color=cl[edge_marker_is_boundary[eidx]],
    )

ax.set_xlim([points[:, 0].min() - 1 / 2, points[:, 0].max() + 1 / 2])
ax.set_ylim([points[:, 1].min() - 1 / 2, points[:, 1].max() + 1 / 2])

fig.tight_layout()
plt.tight_layout()

plt.show()

In [ ]:
total_points = points.shape[0]
num_midpoints = is_middle.sum()
num_nodes = total_points - num_midpoints
NUM = 3*num_nodes + num_midpoints

In [ ]:
x, y = sympy.symbols("x y")

In [ ]:
F = sympy.Float(1)
F_x = F.diff(x)
F_y = F.diff(y)

In [ ]:
right_part_values = np.zeros((total_points, 3))
right_part_values[:, 0] = sympy.lambdify((x, y), F)(*points.T)
right_part_values[:, 1] = sympy.lambdify((x, y), F_x)(*points.T)
right_part_values[:, 2] = sympy.lambdify((x, y), F_y)(*points.T)

In [ ]:
def orient_batch(arg):
    indices = np.argsort(arg[:, :3], axis=-1)
    vertices = np.take_along_axis(arg[:, :3], indices, axis=-1)
    middle_points = np.take_along_axis(arg[:, 3:], indices, axis=-1)
    oriented = np.concatenate([vertices, middle_points], axis=-1)

    return oriented

In [ ]:
triangles = orient_batch(triangles)

In [ ]:
matrix = lil_matrix((NUM, NUM))
b = np.zeros(NUM)

for tidx, element in enumerate(tqdm(triangles)):
    
    x0, x1, x2 = points[element[0], 0], points[element[1], 0], points[element[2], 0]
    y0, y1, y2 = points[element[0], 1], points[element[1], 1], points[element[2], 1]
    
    wff = weak_form_functional_lambdified(0, 0, x0, y0, x1, y1, x2, y2)
    
    wfrp = weak_form_right_part_lambdified(0, 0,
                                            x0, y0, x1, y1, x2, y2,
                                            
                                            right_part_values[element[0], 0],
                                            right_part_values[element[0], 1],
                                            right_part_values[element[0], 2],
                                            
                                            right_part_values[element[1], 0],
                                            right_part_values[element[1], 1],
                                            right_part_values[element[1], 2],
                                            
                                            right_part_values[element[2], 0],
                                            right_part_values[element[2], 1],
                                            right_part_values[element[2], 2],                                            
                                                                                        
                                            right_part_values[element[3], 1],
                                            right_part_values[element[3], 2],
                                            
                                            right_part_values[element[4], 1],
                                            right_part_values[element[4], 2],
                                            
                                            right_part_values[element[5], 1],
                                            right_part_values[element[5], 2],                                            
                                           )
    
    for trial_vertex_idx in range(3):
        
        if vertex_marker_is_boundary[element[trial_vertex_idx]] == 1:
            
            for trial_dof_idx in range(3):      
                     
                matrix[3*element[trial_vertex_idx] + trial_dof_idx, 3*element[trial_vertex_idx] + trial_dof_idx] = 1                
                b[3*element[trial_vertex_idx] + trial_dof_idx] = 0
                    
        else:   
            for trial_dof_idx in range(3):
                for test_vertex_idx in range(3):
                    for test_dof_idx in range(3):
                        matrix[3*element[trial_vertex_idx] + trial_dof_idx, 3*element[test_vertex_idx] + test_dof_idx] += \
                        wff[3*trial_vertex_idx + trial_dof_idx, 3*test_vertex_idx + test_dof_idx]
                        
                for test_edge_idx in range(3, 6):
                    matrix[3*element[trial_vertex_idx] + trial_dof_idx, 3*num_nodes + (element[test_edge_idx] - num_nodes)] += \
                    wff[3*trial_vertex_idx + trial_dof_idx, 9 + (test_edge_idx - 3)]
                
                b[3*element[trial_vertex_idx] + trial_dof_idx] += wfrp[3*trial_vertex_idx + trial_dof_idx, 0]
                                    
                        
    for trial_edge_idx in range(3, 6):
        if vertex_marker_is_boundary[element[trial_edge_idx]] == 1:
        
            matrix[3*num_nodes + (element[trial_edge_idx] - num_nodes), 3*num_nodes + (element[trial_edge_idx] - num_nodes)] = 1
            
            b[3*num_nodes + (element[trial_edge_idx] - num_nodes)] = 0
        else:            
            for test_vertex_idx in range(3):
                for test_dof_idx in range(3):
                    matrix[3*num_nodes + (element[trial_edge_idx] - num_nodes), 3*element[test_vertex_idx] + test_dof_idx] += \
                    wff[9 + (trial_edge_idx - 3), 3*test_vertex_idx + test_dof_idx]
                    
            for test_edge_idx in range(3, 6):
                matrix[3*num_nodes + (element[trial_edge_idx] - num_nodes), 3*num_nodes + (element[test_edge_idx] - num_nodes)] += \
                wff[9 + (trial_edge_idx - 3), 9 + (test_edge_idx - 3)]
            
            b[3*num_nodes + (element[trial_edge_idx] - num_nodes)] += wfrp[9 + (trial_edge_idx - 3), 0]                                

In [ ]:
c = spsolve(matrix.tocsc(), b)

In [ ]:
F =  c[0 : (3 * num_nodes) : 3]
Fx = c[1 : (3 * num_nodes) : 3]
Fy = c[2 : (3 * num_nodes) : 3]
Fn = c[3*num_nodes :]

In [ ]:
param = np.linspace(0, 1, 40)
gridx, gridy = np.meshgrid(param, param, indexing="ij")

mask = (gridy <= 1 - gridx)

gridx = gridx[mask]
gridy = gridy[mask]

In [ ]:
global_x, global_y, global_z = [], [], []

for element in triangles:
    
    x0, x1, x2 = points[element[0], 0], points[element[1], 0], points[element[2], 0]
    y0, y1, y2 = points[element[0], 1], points[element[1], 1], points[element[2], 1]
    
    physical_x, physical_y = mapping_function_lambdified(gridx, gridy, x0, y0, x1, y1, x2, y2).squeeze()
    
    physical_z = interpolation_function_normal(
                                        gridx, gridy,
                                        
                                        x0, y0, 
                                        x1, y1, 
                                        x2, y2,
                                        
                                        F[element[0]], 
                                        Fx[element[0]], 
                                        Fy[element[0]],
                                        
                                        F[element[1]], 
                                        Fx[element[1]], 
                                        Fy[element[1]],
                                        
                                        F[element[2]], 
                                        Fx[element[2]], 
                                        Fy[element[2]],
                                                                                
                                        Fn[element[3] - num_nodes],
                                        Fn[element[4] - num_nodes],                                                                      
                                        Fn[element[5] - num_nodes],
                                        "f_x"
                                        ).squeeze()
    
    global_x.append(physical_x)
    global_y.append(physical_y)
    global_z.append(physical_z)
    
global_x = np.concatenate(global_x)
global_y = np.concatenate(global_y)
global_z = np.concatenate(global_z)

# Plot

In [ ]:
# plot_data = [
#     plotly.graph_objects.Mesh3d(
#                                 x=global_x, 
#                                 y=global_y,
#                                 z=global_z,
#                                 intensity=global_z
#                                 )        
#     ]
# fig = plotly.graph_objects.Figure(data=plot_data)
# fig.update_layout()

# Elementwise plot

In [ ]:
plot_data = [
    plotly.graph_objects.Mesh3d(
                                x=gx, 
                                y=gy,
                                z=gz, 
                                )
        for gx, gy, gz in zip(
                                global_x.reshape(-1, gridx.shape[0]), 
                                global_y.reshape(-1, gridx.shape[0]), 
                                global_z.reshape(-1, gridx.shape[0])
                              )
    ]
fig = plotly.graph_objects.Figure(data=plot_data)
fig.update_layout()